# Semi-supervised Dataset Preparation

- Mục tiêu: tạo bộ dữ liệu giữ cả phần **chưa có nhãn AQI** (aqi_class = NaN) để dùng cho self-training/co-training.
- Đồng thời **giả lập thiếu nhãn trong TRAIN** (time-aware) để mini project có thể thử nhiều mức thiếu nhãn.

In [5]:
import sys
import os
# Thêm đường dẫn gốc của dự án vào hệ thống
sys.path.append(os.path.abspath(os.path.join('..')))

# Bây giờ lệnh import này sẽ chạy được mà không lỗi
from src.semi_supervised_library import SemiDataConfig, mask_labels_time_aware

In [6]:
CLEANED_PATH = "data/processed/cleaned.parquet"
OUTPUT_SEMI_DATASET_PATH = "data/processed/dataset_for_semi.parquet"
CUTOFF = "2017-01-01"
LABEL_MISSING_FRACTION = 0.95
RANDOM_STATE = 42

In [7]:
from pathlib import Path
import pandas as pd

from src.semi_supervised_library import SemiDataConfig, mask_labels_time_aware

PROJECT_ROOT = Path(".").resolve()
if not (PROJECT_ROOT / "data").exists() and (PROJECT_ROOT.parent / "data").exists():
    PROJECT_ROOT = PROJECT_ROOT.parent.resolve()
df = pd.read_parquet((PROJECT_ROOT / CLEANED_PATH).resolve())

cfg = SemiDataConfig(cutoff=CUTOFF, random_state=int(RANDOM_STATE))
df2 = mask_labels_time_aware(df, cfg, missing_fraction=float(LABEL_MISSING_FRACTION))

out_path = (PROJECT_ROOT / OUTPUT_SEMI_DATASET_PATH).resolve()
out_path.parent.mkdir(parents=True, exist_ok=True)
df2.to_parquet(out_path, index=False)

print("Saved:", out_path)
print("Rows:", len(df2))
print("Labeled ratio:", float(df2["is_labeled"].mean()) if "is_labeled" in df2.columns else None)

Saved: C:\Users\PC\air_guard\data\processed\dataset_for_semi.parquet
Rows: 420768
Labeled ratio: 0.08671049129211347


In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import os

# 1. Nạp dữ liệu đã xử lý từ Bước 1
# Đường dẫn '../' là bắt buộc để nhảy ra khỏi thư mục 'notebooks' và vào thư mục 'data'
df = pd.read_parquet('../data/processed/cleaned_data.parquet')

# 2. Lọc dữ liệu trước năm 2017 theo yêu cầu bài tập
# Việc này để đảm bảo tập Test (2017+) hoàn toàn độc lập với quá trình huấn luyện Semi-supervised
train_val_df = df[df['year'] < 2017].copy()

print(f"Nạp dữ liệu thành công!")
print(f"Tổng số mẫu huấn luyện khả dụng (trước 2017): {len(train_val_df)}")

Nạp dữ liệu thành công!
Tổng số mẫu huấn luyện khả dụng (trước 2017): 403776


In [10]:
# 1. Loại bỏ các dòng bị giá trị thiếu trong cột nhãn (aqi_class)
# Việc này đảm bảo stratify có thể hoạt động chính xác
train_val_cleaned = train_val_df.dropna(subset=['aqi_class']).copy()

print(f"Số mẫu sau khi loại bỏ NaN trong nhãn: {len(train_val_cleaned)}")

# 2. Thực hiện tách 10% Labeled và 90% Unlabeled
from sklearn.model_selection import train_test_split

labeled_df, unlabeled_df = train_test_split(
    train_val_cleaned, 
    test_size=0.90, 
    random_state=42, 
    stratify=train_val_cleaned['aqi_class'] # Bây giờ sẽ không còn lỗi
)

# 3. Ẩn nhãn của tập Unlabeled
unlabeled_df_masked = unlabeled_df.copy()
unlabeled_df_masked['aqi_class'] = -1 
unlabeled_df_masked['is_labeled'] = False

# 4. Đánh dấu tập Labeled
labeled_df = labeled_df.copy()
labeled_df['is_labeled'] = True

print(f"Số mẫu có nhãn (10%): {len(labeled_df)}")
print(f"Số mẫu ẩn nhãn (90%): {len(unlabeled_df_masked)}")

Số mẫu sau khi loại bỏ NaN trong nhãn: 403572
Số mẫu có nhãn (10%): 40357
Số mẫu ẩn nhãn (90%): 363215


In [11]:
# Lưu dữ liệu vào thư mục semi_supervised
output_path = '../data/semi_supervised'
import os
os.makedirs(output_path, exist_ok=True)

labeled_df.to_parquet(f'{output_path}/labeled_data.parquet', index=False)
unlabeled_df_masked.to_parquet(f'{output_path}/unlabeled_data.parquet', index=False)

print("--- ĐÃ LƯU DỮ LIỆU THÀNH CÔNG! ---")

--- ĐÃ LƯU DỮ LIỆU THÀNH CÔNG! ---
